In [1]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.modeling import BertForSequenceClassification
from pathlib import Path
import torch

import pandas as pd
import sys
import random
import numpy as np
import os
from tqdm import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader, SequentialSampler

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [2]:
MODEL_PATH=Path('../models/intent_classification_lib_2019-02-17_23-26-25.bin')
BERT_PRETRAINED_PATH = Path('../../bert_fastai/pretrained-weights/uncased_L-12_H-768_A-12/')
LABEL_PATH = Path('../labels/labels.csv')

In [3]:
pd.set_option('display.max_colwidth', -1)

In [4]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

In [5]:
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b)
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[:(max_seq_length - 2)]

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = ["[CLS]"] + tokens_a + ["[SEP]"]
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens += tokens_b + ["[SEP]"]
            segment_ids += [1] * (len(tokens_b) + 1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        
        if example.label:
            label_id = label_map[example.label]
        else:
            label_id = ''

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features

In [6]:
def predict_batch(model, label_list, texts, max_seq_length=512, batch_size=32, device='cuda'):
    
    examples = []
    input_data = []
    
    for index, text in enumerate(texts):
        examples.append(InputExample(index, text))
        input_data.append({
            'id': index, 
            'text': text
        })
           
    test_features = convert_examples_to_features(examples, label_list, max_seq_length, tokenizer)
    
    
    all_input_ids = torch.tensor([f.input_ids for f in test_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in test_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in test_features], dtype=torch.long)

    test_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids)
    
    # Run prediction for full data
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    all_logits = None
    
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for step, batch in enumerate(test_dataloader):
        input_ids, input_mask, segment_ids = batch
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)

        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask)
            logits = logits.softmax(dim=1)

        if all_logits is None:
            all_logits = logits.detach().cpu().numpy()
        else:
            all_logits = np.concatenate((all_logits, logits.detach().cpu().numpy()), axis=0)
            
        nb_eval_examples += input_ids.size(0)
        nb_eval_steps += 1

    result_df =  pd.DataFrame(all_logits, columns=label_list)
    results = result_df.to_dict('record')
    
    return [sorted(x.items(), key=lambda kv: kv[1], reverse=True) for x in results]

In [7]:
def load_model(device='cuda'):
    labels = list(pd.read_csv(LABEL_PATH,header=None).dropna()[0].values)
    
    # Load a trained model that you have fine-tuned
    model_state_dict = torch.load(MODEL_PATH)
    model = BertForSequenceClassification.from_pretrained(BERT_PRETRAINED_PATH, num_labels = len(labels), state_dict=model_state_dict)
    model.half()
    model.to(device)
    return labels, model
    

In [36]:
labels, model = load_model()

In [37]:
tokenizer = tokenizer = BertTokenizer.from_pretrained(BERT_PRETRAINED_PATH, do_lower_case=True)

In [241]:
text = """
i would like to change my service visit
"""
predict_batch(model, labels, [text])

[[('change-appointment', 0.5312),
  ('change-appointment_service', 0.4011),
  ('change-payment', 0.00936),
  ('change-direct_debit', 0.007435),
  ('enquire-meter_exchange', 0.006138),
  ('change-appointment_boiler', 0.006042),
  ('change-bill', 0.00442),
  ('change-vague', 0.003338),
  ('change-homecare', 0.0029),
  ('change-account_detail', 0.001684),
  ('cancel-appointment', 0.001464),
  ('request-appointment', 0.001204),
  ('change-supplier', 0.001204),
  ('enquire-appointment_when', 0.001131),
  ('enquire-appointment_smart_meter', 0.0010395),
  ('change-phone', 0.001024),
  ('problem-drain', 0.0008893),
  ('change-address', 0.000755),
  ('enquire-appointment', 0.000652),
  ('enquire-service', 0.0005937),
  ('vague-appliance', 0.000531),
  ('report-bill_high', 0.0004702),
  ('change-install_smart_meter', 0.000428),
  ('request-card', 0.000424),
  ('request-service', 0.0004077),
  ('report-direct_debit_changed', 0.0004053),
  ('enquire-business_care', 0.0003307),
  ('request-install_

In [34]:
tmp_labels.sort()
tmp_labels

['None',
 'cancel-appointment',
 'cancel-business_account',
 'cancel-contract',
 'cancel-direct_debit',
 'cancel-energy',
 'cancel-hive',
 'cancel-smart_meter',
 'cancel-tariff',
 'cancel-vague',
 'change-account_detail',
 'change-address',
 'change-appointment',
 'change-appointment_boiler',
 'change-appointment_service',
 'change-bill',
 'change-direct_debit',
 'change-email',
 'change-homecare',
 'change-install_smart_meter',
 'change-name',
 'change-ownership',
 'change-payment',
 'change-payment_plan',
 'change-phone',
 'change-supplier',
 'change-vague',
 'delete-delete',
 'dispute-bill',
 'enquire-account',
 'enquire-account_confirm',
 'enquire-account_online',
 'enquire-account_overdue',
 'enquire-activation_code',
 'enquire-agreement',
 'enquire-alarm',
 'enquire-annual_service',
 'enquire-appointment',
 'enquire-appointment_boiler',
 'enquire-appointment_plumbing',
 'enquire-appointment_smart_meter',
 'enquire-appointment_today',
 'enquire-appointment_when',
 'enquire-authori

In [242]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertInterme